In [1]:
#Import the libraries

import pandas as pd
import numpy as np
import json
import nltk
import re
import csv
import matplotlib.pyplot as plt 
import seaborn as sns
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, AutoConfig, GPTNeoForCausalLM, AutoTokenizer
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader, random_split
import os
import openai
from re import search
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity

/work/anaseh_umass_edu/anaconda3/envs/test/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.cuda.device_count()

1

In [ ]:
device=torch.device("cuda")

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/anaseh_umass_edu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
# Modify the tokenizer if needed.

tokenizer = AutoTokenizer.from_pretrained("gpt2-medium")

In [5]:
# Change the config if needed.
config = AutoConfig.from_pretrained("gpt2-medium")

# Change the tokenizer if needed.
model = AutoModelForCausalLM.from_pretrained("gpt2-medium", config=config, pad_token_id=tokenizer.eos_token_id).cuda()
model.resize_token_embeddings(len(tokenizer))

In [ ]:
# Modify the path if needed
model.load_state_dict(torch.load('gpt_2_test/checkpoint-100/pytorch_model.bin'))

In [6]:
model_similarity_check = SentenceTransformer('all-MiniLM-L6-v2')

In [7]:
def similarity_score(text1, text2):
    
    #Compute embedding for both lists
    embeddings1 = model_similarity_check.encode(text1, convert_to_tensor=True)
    embeddings2 = model_similarity_check.encode(text2, convert_to_tensor=True)
    
    #Compute cosine-similarities
    cosine_scores = util.cos_sim(embeddings1, embeddings2)
    
    return cosine_scores

In [ ]:
"""
If the decoding algorithm is one of the greedy search or beam search, the generated text might be repetitive. 
Hence, we wrote this function to check if the generated text is repetitive. If so, we will remove that sample.
"""

def has_repetition(generated_text, window_size=10):
    """
    Determines if there is repetitive text in a generated text using a sliding window of the specified size.
    Returns True if there is any repetition and False otherwise.
    """
    window_start = 0
    window_end = window_size
    while window_end < len(generated_text):
        window_text = generated_text[window_start:window_end]
        if window_text in generated_text[window_end:]:
            return True
        window_start += 1
        window_end += 1
    return False


In [ ]:
"""
We initiate a dictionary to save values for different lengths. For each length, we save the number of samples of that length and the sum of the scores. In the end, we can compute the average score for each length.
"""

results = {}
for i in range(100):
    results[str(i)] = [0,0]

In [8]:
# Import the dataset
df = pd.read_csv('train.csv')

In [9]:
df.head(4)

,Unnamed: 0,text
0,0,<cs.CR>: An access control model called Zero...
1,1,"<gr-qc>: Using quantum Riemannian geometry, ..."
2,2,<gr-qc>: Noether's theorem identifies fundam...
3,3,<stat.ME>: Multilevel models (mixed-effect m...


In [ ]:

flags = [1000, 4000, 8000, 15000, 27000]
num_of_repetitive_samples = 0

for i in range(len(df)):
    
    if i in flags:
        print(i)

    
    # You can change the length to test different values.    
    prompt = df['text'][i].split(" ")[:50]
    prompt = ' '.join(prompt)
    init_sentences = nltk.sent_tokenize(prompt)
    length = len(init_sentences)
    input_ids = tokenizer(prompt, return_tensors='pt').to(device)
    
    # You can modify the max_length based on the prompt length.
    temp_text = model.generate(**input_ids, max_length=150, num_beams = 6)
    temp_text = tokenizer.decode(temp_text[0], skip_special_tokens=True)
    
   
    sentences = nltk.sent_tokenize(temp_text)
    final_text = sentences[length-1]
    
    if has_repetition(final_text):
        #print(final_text)
        num_of_repetitive_samples += 1
        continue
    
    orig_sentences = nltk.sent_tokenize(df['text'][i])
    score = similarity_score(orig_sentences[length-1], final_text)
    diff = len(final_text.split(' ')) - len(init_sentences[length-1].split(" "))
    
    
    results[str(diff)][0] += 1
    results[str(diff)][1] += score[0].item()
    #print("Score: " + str(score))
    #print("Diff: " + str(diff))
    #print("------------------------------------")

In [40]:
for i in range(len(results)):
    if results[str(i)][0] > 0:
        results[str(i)][1] = results[str(i)][1]/results[str(i)][0]
    

In [42]:
print(results)

{'0': [5, 0.8857958436012268], '1': [3, 0.9013360540072123], '2': [6, 0.722661112745603], '3': [4, 0.6346058398485184], '4': [7, 0.775366987500872], '5': [2, 0.6704206466674805], '6': [2, 0.48442715406417847], '7': [6, 0.803336888551712], '8': [4, 0.5602045580744743], '9': [0, 0], '10': [1, 0.8796998262405396], '11': [2, 0.6954313218593597], '12': [0, 0], '13': [3, 0.41146397590637207], '14': [2, 0.5290040820837021], '15': [3, 0.46533167362213135], '16': [2, 0.370731383562088], '17': [4, 0.5017414093017578], '18': [1, 0.36304786801338196], '19': [2, 0.4280588924884796], '20': [1, 0.3395509123802185], '21': [1, 0.33076173067092896], '22': [1, 0.6550519466400146], '23': [0, 0], '24': [0, 0], '25': [0, 0], '26': [0, 0], '27': [0, 0], '28': [0, 0], '29': [0, 0], '30': [0, 0], '31': [0, 0], '32': [0, 0], '33': [0, 0], '34': [0, 0], '35': [0, 0], '36': [0, 0], '37': [0, 0], '38': [0, 0], '39': [0, 0], '40': [0, 0], '41': [0, 0], '42': [0, 0], '43': [0, 0], '44': [0, 0], '45': [0, 0], '46': [

In [43]:
df_results = pd.DataFrame.from_dict(results, orient='index', columns=['Number', 'Mean'])

# Set the name of saved file in the same format of the fine-tuned model name
df_results.to_csv('test.csv', index_label='key')